In [1]:
#Set up the environment
library(tidyverse)
library(reshape2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [ ]:
#Get a list of all of the somatic files
setwd("../tcga_somatic")
somatic.vcf.file.list = list.files(path = ".", pattern = "avana.filtered")

In [ ]:
#Loop through all of the files, load them, filter to passing variants, then calculate the number of variants
num.somatic.variants.in.avana.guides.vector = NULL
for(file in somatic.vcf.file.list){
    
    num.variants = read.table(file, sep = "\t") %>% filter(V7 %in% "PASS") %>% nrow() #Load the file and calculate the number of rows (which corresponds to the number of SNVs)
    num.somatic.variants.in.avana.guides.vector = c(num.somatic.variants.in.avana.guides.vector, num.variants) #Add this number to the vector outside of the loop
}

In [2]:
#Load in the sample information sheet for the vcf files
setwd("../tcga_somatic")
somatic.vcf.sample.information.sheet = read.table("vcf_sample_sheet.tsv", sep = "\t", header = T) %>%
select(File.ID, File.Name,  Project.ID, Sample.ID, Sample.Type) %>%
rename("file_id" = 1, "file_name" = 2, "project_id" = 3, "sample_id" = 4, "sample_type" = 5) %>%
separate(sample_type,  sep = ", ", into = c("sample_type_1", "sample_type_2")) %>%
separate(sample_id, sep = ", ", into = c("sample_id_1", "sample_id_2")) %>%
mutate("tumor_id" = ifelse(sample_type_1 %in% "Primary Tumor", sample_id_1, 
                          ifelse(sample_type_2 %in% "Primary Tumor", sample_id_2, NA))) %>%

mutate("normal_id" = ifelse(sample_type_1 %in% "Primary Tumor", sample_id_2, 
                          ifelse(sample_type_2 %in% "Primary Tumor", sample_id_1, NA))) %>%

select(file_id, file_name, project_id, tumor_id, normal_id)

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 10 rows [1023, 1032,
1039, 1046, 6238, 6240, 6241, 6250, 6291, 10008].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 10 rows [1023, 1032,
1039, 1046, 6238, 6240, 6241, 6250, 6291, 10008].”


In [ ]:
#Assemble a data frame with the somatic variants 
somatic.variant.in.avana.guide.df = cbind(somatic.vcf.file.list, num.somatic.variants.in.avana.guides.vector) %>%
data.frame() %>%
rename("file_name" = 1, "num_somatic_var" = 2) %>%
mutate(num_somatic_var = as.numeric(num_somatic_var)) %>%
mutate(file_name = gsub("avana.filtered.", "", file_name)) %>%
inner_join(somatic.vcf.sample.information.sheet, by = "file_name")

head(somatic.variant.in.avana.guide.df)

In [ ]:
#Load in the sample names
setwd("../tcga_germline")
germline.sample.names = read.table("tcga_germline_sample_names.txt", sep = "\t") %>% pull(V1)

#Load the germline data into R
setwd("../tcga_germline")
tcga.germline.mutations.in.avana.guides = read.table("tcga_germline_variants_in_avana_guides.vcf.gz", sep = "\t")

In [ ]:
#Format the dataset so that it is in a format that makes it easy to count
num.germline.variants.per.sample = tcga.germline.mutations.in.avana.guides %>%
dplyr::select(-V1, -V2, -V3, -V4, -V5, -V6, -V7, -V8, -V9) %>%
mutate(across(everything(), gsub, pattern = ":..*", replacement = "")) #Fix all of the encodings for the variants

In [ ]:
#Format the dataset so that it is in a format that makes it easy to count
num.germline.variants.per.sample = tcga.germline.mutations.in.avana.guides %>%
dplyr::select(-V1, -V2, -V3, -V4, -V5, -V6, -V7, -V8, -V9) %>%
mutate(across(everything(), gsub, pattern = ":..*", replacement = "")) #Fix all of the encodings for the variants

In [ ]:
#Create and write a data frame that has the number of germline variants per sample
germline.variants.per.sample.df = cbind(germline.sample.names, variants.per.sample) %>%
data.frame() %>%
rename("sample" = 1, "num_germline_variants" = 2)

#Write the data frame
setwd("../output")
write.table(germline.variants.per.sample.df, "figure_3d_germline.txt", sep = "\t", col.names = TRUE, row.names = FALSE, quote = FALSE)

In [ ]:
#Format the germline.variants.per.sample.df dataset so that it is in the correct format for merging with the somatic data
processed.germline.variants.per.sample.df = germline.variants.per.sample.df %>%
separate(sample, sep = "-", into = c("sample_1", "sample_2", "sample_3", "sample_4", "sample_5", "sample_6", "sample_7")) %>% #I don't know how to use regex
mutate(sample = paste(sample_1, "-", sample_2, "-", sample_3, "-", sample_4, sep = ""), .before = 1) %>% #I don't know how to use regex
select(-sample_1, -sample_2, -sample_3, -sample_4, -sample_5, -sample_6, -sample_7) #I don't know how to use regex

In [ ]:
#Merge the germline and the somatic datasets together
merged.tcga.germline.somatic.in.avana = processed.germline.variants.per.sample.df %>%
rename("normal_id" = sample) %>%
inner_join(somatic.variant.in.avana.guide.df, by = "normal_id") %>%
mutate(project_id = gsub("TCGA-", "", project_id))

In [ ]:
#Calculate the mean number of somatic mutations per cancer type
somatic.mutations.per.cancer.type = merged.tcga.germline.somatic.in.avana %>%
group_by(project_id) %>% 
summarise("num_somatic" = median(num_somatic_var)) %>%
arrange(num_somatic) %>%
mutate(project_id = factor(project_id))

In [ ]:
#Plot the plot
merged.tcga.germline.somatic.in.avana %>%
mutate(project_id = factor(project_id, levels = somatic.mutations.per.cancer.type$project_id)) %>%
ggplot() +
geom_boxplot(aes(x = project_id, y = log10(num_germline_variants)), color = "firebrick") +
geom_boxplot(aes(x = project_id, y = log10(num_somatic_var)), color = "grey") +

coord_flip() +

theme_bw() +


theme(
panel.grid.major = element_blank(),
panel.grid.minor = element_blank(), 
) +

theme(
axis.title.x = element_text(color = "black", size = 12),
axis.title.y = element_text(color = "black", size = 12)
) +

theme(
axis.text = element_text(size = 12),
axis.title = element_text(size = 12)
) +

theme(
legend.title = element_text(size = 12),
legend.text = element_text(size = 12)
) +

ylab("Number of variants in Avana guides") +
xlab("Cancer type")

#Export it to the google bucket
setwd('../output')
ggsave("figure_3d.pdf", width = 5, height = 6)

In [ ]:
#Also write the output df
df_for_output = merged.tcga.germline.somatic.in.avana %>%
mutate(project_id = factor(project_id, levels = somatic.mutations.per.cancer.type$project_id))

setwd("../output")
write.table(df_for_output, "figure_3d_germline_somatic.txt", sep = "\t", col.names = TRUE, row.names = FALSE, quote = FALSE)